In [1]:
from data import load_data, resampling, csvDS, test_split
from models import MLP
from torch import nn
from torch.optim import SGD
import torch

In [2]:
# test all sampling methods
for dsname in ['vehicle', 'diabete', 'vowel', 'ionosphere', 'abalone']:
    x, y = load_data(dsname)
    print('='*20, dsname, '='*20)
    print('original:', x.shape, y.shape, y.sum(), x.dtype, y.dtype)
    for sampling in ['nonsampling', 'oversampling', 'smote', 'adasyn', 'blmovgen', 'admovgen', 'adboth']:
    # for sampling in ['nonsampling', 'oversampling', 'smote', 'adasyn', 'blsmote', 'adsmote', 'adboth']:
        x_resampl, y_resampl = resampling(x, y, sampling=sampling)
        print(sampling, x_resampl.shape, y_resampl.shape, y_resampl.sum(), x_resampl.dtype, y_resampl.dtype)



==================== vehicle ====================
original: (846, 18) (846,) 199.0 float32 float32
nonsampling (846, 18) (846,) 199.0 float32 float32
oversampling (1294, 18) (1294,) 647.0 float32 float32
smote (1294, 18) (1294,) 647.0 float32 float32
adasyn (1296, 18) (1296,) 649.0 float32 float32
blmovgen (1294, 18) (1294,) 647.0 float32 float64
admovgen (1294, 18) (1294,) 647.0 float32 float64
adboth (1612, 18) (1612,) 806.0 float32 float64
==================== diabete ====================
original: (768, 8) (768,) 268.0 float32 float32
nonsampling (768, 8) (768,) 268.0 float32 float32
oversampling (1000, 8) (1000,) 500.0 float32 float32
smote (1000, 8) (1000,) 500.0 float32 float32
adasyn (974, 8) (974,) 474.0 float32 float32
blmovgen (1000, 8) (1000,) 500.0 float32 float64
admovgen (1000, 8) (1000,) 500.0 float32 float64
adboth (2036, 8) (2036,) 1018.0 float32 float64
==================== vowel ====================
original: (990, 10) (990,) 90.0 float32 float32
nonsampling (990, 1

In [4]:
# training workflows

# define the parameters
dsname = 'vehicle'
sampling = 'adasyn'
normalize = 'scale'
seed = 0
test_size = 0.2

# load data
x, y = load_data(dsname)
x_train, x_test, y_train, y_test = test_split(x, y, test_size=test_size, seed=seed)
x_train, y_train = resampling(x_train, y_train, sampling=sampling, normalize=normalize, seed=seed)

# convert to dataset
train_ds = csvDS(x_train, y_train)
test_ds = csvDS(x_test, y_test)

# define the model
model = MLP(input_size=x_train.shape[1])

# define the loss function
loss_fn = nn.BCEWithLogitsLoss()
optimizer = SGD(model.parameters(), lr=1e-3)

# train the model
for t in range(40):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(train_ds.x)

    # Compute and print loss.
    loss = loss_fn(y_pred, train_ds.y)
    if t % 10 == 9:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# test the model
with torch.no_grad():
    y_pred = model(test_ds.x)
    y_pred = torch.sigmoid(y_pred)
    y_pred = (y_pred > 0.5).float()
    print('test acc:', (y_pred == test_ds.y).float().mean().item())


9 0.6916780471801758
19 0.6903680562973022
29 0.6892580389976501
39 0.6883236169815063
test acc: 0.7928994297981262
